In [ ]:
from requests import exceptions

import matplotlib.pyplot as plt
import requests
import argparse
import cv2
import os
import json


In [ ]:

# ap = argparse.ArgumentParse()
# ap.add_argument("-q", "--query", required=True,
#                help="search query to search Bing Image API for")
# ap.add_argument("-o", "--output", required=True,
#                help="path to output of images")
# args = vars(ap.parse_args())

args = {
    "query": "cat",
    "output": "dataset/test"
}

In [ ]:
API_KEY    = "e8c37798f5msh3238c1b684e4791p111ee5jsnf66f0346e534"
URL        = "https://bing-image-search1.p.rapidapi.com/images/search"

MAX_RESULT = 20
GROUP_SIZE = 5

search_term= args["query"]
headers    = {
    "x-rapidapi-key": API_KEY
}
params    = {
    "q"     : search_term,
    "offset": 0,
    "count" : GROUP_SIZE    
}

print("[INFO] searching Bing API for '{}'".format(search_term))

search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()

results = search.json()
estNumResults = min(results["totalEstimatedMatches"], MAX_RESULT)

print("[INFO] total {} reslts for '{}'".format(estNumResults, search_term))



In [ ]:
total = 0

for offset in range(0, estNumResults, GROUP_SIZE):
    print("[INFO] making request for group {}-{} of {}...".format(offset, offset+GROUP_SIZE, estNumResults))
    
    params["offset"] = offset
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    results = search.json()
    
    print("[INFO] saving images for group {}-{} of {}...".format(offset, offset+GROUP_SIZE, estNumResults))
    


In [ ]:


EXCEPTIONS = set([IOError, FileNotFoundError, 
                  exceptions.RequestException, exceptions.HTTPError, 
                  exceptions.ConnectionError, exceptions.Timeout])

for v in results["value"]:
    try:
        print("[INFO] feaching: {}".format(v["contentUrl"]))
        
        r = requests.get(v["contentUrl"], timeout=30)
        
        ext= v["contentUrl"][v["contentUrl"].rfind("."):]
        p = os.path.sep.join([args["output"], "{}{}".format(str(total).zfill(8), ext)])
        
        f = open(p, "wb")
        f.write(r.content)
        f.close()
        
    except Exception as e:
        if type(e) in EXCEPTIONS:
            print("[INFO] skipping: {}".format(v["contentUrl"]))
        else:
            print("[INFO] skipped")
            
print("[INFO] done!!! ")